# Деревья решений

## Построение дерева

Опишем жадный алгоритм построения бинарного дерева решений:
1. Начинаем со всей обучающей выборки $X$, которую помещаем в корень $R_1$.
2. Задаём функционал качества $Q(X, j, t)$ и критерий остановки.
3. Запускаем построение из корня: $SplitNode(1, R_1)$

Функция $SplitNode(m, R_m)$
1. Если выполнен критерий остановки, то выход.
2. Находим наилучший с точки зрения $Q$ предикат: $j, t$: $[x_j<t]$
3. Помещаем предикат в вкршину и получаем с его помощью разбиение $X$ на две части: $R_{left} = \lbrace x|x_j<t \rbrace$ и $R_{right} = \lbrace x|x_j \geqslant t \rbrace$
4. Поместим $R_{left}$ и $R_{right}$ соответсвенно в левое и правое поддерево.
5. Рекурсивно повторяем $SplitNode(left, R_{left})$ и $SplitNode(right, R_{right})$.

В конце поставим в соответствие каждому листу ответ. Для задачи классификации - это самый частый среди объектов класс или вектор с долями классов (можно интерпретировать как вероятности):
$$ c_v = \arg \max_{k\in Y} \sum_{(x_i,y_i) \in R_v} [y_i=k]  $$

## Функционал качества для деревьев решений


Энтропия Шеннона для системы с N возможными состояниями определяется по формуле:
$$H = - \sum_{i=0}^{N} p_i\log_2p_i $$

где $p_i$ – вероятности нахождения системы в $i$-ом состоянии.

Это очень важное понятие теории информации, которое позволяет оценить количество информации (степень хаоса в системе). Чем выше энтропия, тем менее упорядочена система и наоборот. С помощью энтропии мы формализуем функционал качества для разделение выборки (для задачи классификации).

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

import random
from pprint import pprint

Код для расчёта энтропии:

In [ ]:
def entropy(y):

    _, counts = np.unique(y, return_counts=True)

    probabilities = counts / counts.sum()
    entropy = sum(probabilities * -np.log2(probabilities))

    return entropy

Здесь $y$ - это массив значений целевой переменной

Энтропия – по сути степень хаоса (или неопределенности) в системе. Уменьшение энтропии называют приростом информации (information gain, IG).

Обочначим $R_v$ - объекты, которые нужно разделить в помощью предиката в вершине $v$. Запишем формулу для расчёта информационного прироста:
$$ Q = IG = H(R_v) - (H(R_{left})+H(R_{right}))$$

На каждом шаге нам нужно максимизировать этот функционал качества. Как это делать? Например, так можно перебрать $t$ для выбранного $j$.

Предыдущая версия формулы прироста информации слишком упрощена. В работе необходимо использовать более устойчивую формулу, которая учитывает не только энтропию подмножеств, но и их размер.

$$ Q = IG = H(R_v) - \Big (\frac{|R_{left}|} {|R_{v}|} H(R_{left})+ \frac{|R_{right}|} {|R_{v}|} H(R_{right})\Big)$$

где, $|R_{v}|$, $|R_{left}|$ и $|R_{right}|$ - количество элементов в соответствующих множествах.


### Задание 4.1

Реализуйте алгоритм построения дерева. Должны быть отдельные функции (методы) для расчёта энтропии (уже есть), для разделения дерева (используйте `pandas`), для подсчёта функционала качества $IG$, для выбора наилучшего разделения (с учетом признакоd и порогов), для проверки критерия остановки.

Для набора данных `iris` реализуйте алгоритм и минимум три из разными критерия остановки из перечисленных ниже:
* максимальной глубины дерева = 5
* минимального числа объектов в листе = 5
* максимальное количество листьев в дереве = 5
* purity (остановка, если все объекты в листе относятся к одному классу)

Реализуйте функцию `predict` (на вход функции подаётся датафрейм с объектами)

Оцените точность каждой модели с помощью метрики доля правильных ответов (`from sklearn.metrics import accuracy_score` или реализовать свою).

Реализация алгоритма построения дерева:

In [ ]:
class TreeClassifier:

  def __init__(self, max_depth=5, min_list_amount=5, max_samples_leaf=5):
    self.depth = 0
    self.leaf = 0
    self.max_depth = max_depth
    self.min_list_amount = min_list_amount
    self.max_samples_leaf = max_samples_leaf
    self.different_y = None

  def entropy(self, y):

    _, counts = np.unique(y, return_counts=True)

    probabilities = counts / counts.sum()
    entropy = sum(probabilities * -np.log2(probabilities))

    return entropy

  def get_params(self, X, y):

    best_IG = -1
    best_t = X[X.columns[0]].min()
    best_col = X.columns[0]

    for col in X.columns:
      grid = np.linspace(X[col].min(), X[col].max(), 10)

      for t in grid:
        sample1 = y[X[col] < t]
        sample2 = y[X[col] >= t]
        IG = self.entropy(y) - (len(sample1) / len(X) * self.entropy(sample1) + len(sample2) / len(X) * self.entropy(sample2))

        if IG > best_IG and not self.min_size_criteria(sample1) and not self.min_size_criteria(sample2):
          best_t, best_IG, best_col  = t, IG, col

    return best_IG, best_t, best_col

  def depth_criteria(self):
    return self.depth == self.max_depth

  def min_size_criteria(self, l):
    return len(l) < self.min_list_amount

  def max_leaf_criteria(self):
    return self.leaf == self.max_samples_leaf

  def purity_criteria(self, y):
    return len(set(y)) == 1

  def fit(self, X, y):

    self.depth += 1

    IG, t, col = self.get_params(X, y)

    y_left = y[X[col] < t]
    y_right = y[X[col] >= t]

    self.different_y = set(y) if not self.different_y else self.different_y

    par_node = {'title': f'{col} < {t}',
                'col': col,
                't':t,
                'value': [(y == i).sum() for i in self.different_y],
                'class': np.round(np.mean(y))}

    # критерии выхода
    if self.purity_criteria(y) | self.depth_criteria() | self.max_leaf_criteria() | self.min_size_criteria(y_left) | self.min_size_criteria(y_right):
      self.leaf += 1
      return {'value': [(y == i).sum() for i in self.different_y], 'class': np.round(np.mean(y))}

    par_node['left'] = self.fit(X[X[col] < t], y_left)
    par_node['right'] = self.fit(X[X[col] >= t], y_right)

    self.tree_ = par_node

    return par_node

  def predict(self, X):
    results = np.array([0]*len(X))
    cols = list(X.columns)

    for i, row in enumerate(X.values):
      cur_layer = self.tree_
      while cur_layer.get('t'):
        if row[cols.index(cur_layer['col'])] < cur_layer['t']:
            cur_layer = cur_layer['left']
        else:
            cur_layer = cur_layer['right']
      else:
          results[i] = cur_layer.get('class')
    return results

Загрузка данных:

In [ ]:
from sklearn import datasets

iris = datasets.load_iris()

df= pd.DataFrame(data= np.c_[iris['data'], iris['target']],
                 columns= iris['feature_names'] + ['target'])

X = df.drop('target', axis=1)
y = df['target']

df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0.0
1,4.9,3.0,1.4,0.2,0.0
2,4.7,3.2,1.3,0.2,0.0
3,4.6,3.1,1.5,0.2,0.0
4,5.0,3.6,1.4,0.2,0.0


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [ ]:
clf = TreeClassifier()
clf.fit(X_train, y_train)
pprint(clf.tree_)

{'class': 1.0,
 'col': 'petal length (cm)',
 'left': {'class': 0.0, 'value': [36, 0, 0]},
 'right': {'class': 2.0,
           'col': 'petal length (cm)',
           'left': {'class': 1.0, 'value': [0, 27, 0]},
           'right': {'class': 2.0, 'value': [0, 4, 38]},
           't': 4.733333333333333,
           'title': 'petal length (cm) < 4.733333333333333',
           'value': [0, 31, 38]},
 't': 2.311111111111111,
 'title': 'petal length (cm) < 2.311111111111111',
 'value': [36, 31, 38]}


Оценка качества алгоритма:

In [ ]:
from sklearn.metrics import accuracy_score

y_pred = clf.predict(X_test)

accuracy_score(y_test, y_pred)

0.9333333333333333

##  Случайный лес

Опишем алгоритм случайный лес (*random forest*) и попутно разберём основные идеи:

1. Зададим $N$ - число деревьев в лесу.
2. Для каждого $n$ из $N$ сгенерируем свою выборку $X_n$. Пусть $m$ - это количество объектов в $X$. При генерации каждой $X_n$ мы будем брать объекты $m$ раз с возвращением. То есть один и тот же объект может попасть в выборку несколько раз, а какие-то объекты не попадут. (Этот способ назвается бутстрап).
3. По каждой $X_n$ построим решающее дерево $b_n$. Обычно стараются делать глубокие деревья. В качестве критериев остановки можно использовать `max_depth` или `min_samples_leaf` (например, пока в каждом листе не окажется по одному объекту). При каждом разбиении сначала выбирается $k$ (эвристика $k = \sqrt d$, где $d$ - это число признаков объектов из выборки $X$) случайных признаков из исходных, и оптимальное разделение выборки ищется только среди них. Обратите внимание, что мы не выбрасываем оставшиеся признаки!
4. Итоговый алгоритм будет представлять собой результат голосования (для классификации) и среднее арифметическое (для регрессии). Модификация алгоритма предполагает учёт весов каждого отдельного слабого алгоритма в ансамбле, но в этом особо нет смысла.


### Задание 4.2

В качестве набора данных используйте: https://www.kaggle.com/mathchi/churn-for-bank-customers

Там есть описание и примеры работы с этими данными. Если кратко, речь идёт про задачу прогнозирования оттока клиентов. Есть данные о 10 тысячах клиентов банка, часть из которых больше не являются клиентами.

Используя либо свою реализацию, либо  `DecisionTreeClassifier` с разными настройками из `sklearn.tree` реализйте алгоритм "случайный лес".

Найдите наилучшие гиперпараметры этого алгоритма: количество деревьев, критерий остановки, функционал качества, минимальное количество объектов в листьях и другие.

Нельзя использовать готовую реализацию случайного леса из `sklearn`.

В подобных задачах очень важна интерпретируемость алгоритма. Попытайтесь оценить информативность признаков, т.е. ответить а вопрос, значения каких признаков являются самыми важными индикаторами того, что банк потеряет клиента.

In [ ]:
! pip install -q kaggle

In [ ]:
from google.colab import files

In [ ]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d mathchi/churn-for-bank-customers --force
! unzip churn-for-bank-customers.zip -d data

mkdir: cannot create directory ‘/root/.kaggle’: File exists
100% 261k/261k [00:00<00:00, 804kB/s]
100% 261k/261k [00:00<00:00, 803kB/s]
Archive:  churn-for-bank-customers.zip
replace data/churn.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: data/churn.csv          


In [ ]:
data = pd.read_csv('data/churn.csv')

In [ ]:
trash_cols = ['CustomerId', 'Surname', 'RowNumber']

data = data.drop(trash_cols, axis=1)

data['Gender'] = np.where(data['Gender'] == 'Female', 1, 0)

In [ ]:
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,1,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,1,41,1,83807.86,1,0,1,112542.58,0
2,502,France,1,42,8,159660.80,3,1,0,113931.57,1
3,699,France,1,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,1,43,2,125510.82,1,1,1,79084.10,0


#### Label Encoder

In [ ]:
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
geog_label = labelencoder.fit_transform(data['Geography'].values)
data_label = data.copy()
data_label['Geography'] = geog_label

In [ ]:
data_label.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,0,1,42,2,0.00,1,1,1,101348.88,1
1,608,2,1,41,1,83807.86,1,0,1,112542.58,0
2,502,0,1,42,8,159660.80,3,1,0,113931.57,1
3,699,0,1,39,1,0.00,2,0,0,93826.63,0
4,850,2,1,43,2,125510.82,1,1,1,79084.10,0


In [ ]:
X_label = data_label.drop('Exited', axis = 1)
y_label = data_label['Exited']

In [ ]:
X_label

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,0,1,42,2,0.00,1,1,1,101348.88
1,608,2,1,41,1,83807.86,1,0,1,112542.58
2,502,0,1,42,8,159660.80,3,1,0,113931.57
3,699,0,1,39,1,0.00,2,0,0,93826.63
4,850,2,1,43,2,125510.82,1,1,1,79084.10
...,...,...,...,...,...,...,...,...,...,...
9995,771,0,0,39,5,0.00,2,1,0,96270.64
9996,516,0,0,35,10,57369.61,1,1,1,101699.77
9997,709,0,1,36,7,0.00,1,0,1,42085.58
9998,772,1,0,42,3,75075.31,2,1,0,92888.52


#### One Hot

In [ ]:
data['Geography']

0        France
1         Spain
2        France
3        France
4         Spain
         ...   
9995     France
9996     France
9997     France
9998    Germany
9999     France
Name: Geography, Length: 10000, dtype: object

In [ ]:
from sklearn.preprocessing import OneHotEncoder
onehotencoder = OneHotEncoder()
geog_onehot = pd.DataFrame(onehotencoder.fit_transform(data[['Geography']].values.reshape(-1,1)).toarray(), columns = ['g1', 'g2', 'g3'])
data_onehot = data.copy().drop('Geography', axis=1)

In [ ]:
geog_onehot

,g1,g2,g3
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [ ]:
data_onehot = data_onehot.join(geog_onehot)

In [ ]:
data_onehot

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,g1,g2,g3
0,619,1,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,1,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,1,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,1,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,1,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,0,39,5,0.00,2,1,0,96270.64,0,1.0,0.0,0.0
9996,516,0,35,10,57369.61,1,1,1,101699.77,0,1.0,0.0,0.0
9997,709,1,36,7,0.00,1,0,1,42085.58,1,1.0,0.0,0.0
9998,772,0,42,3,75075.31,2,1,0,92888.52,1,0.0,1.0,0.0


In [ ]:
X_onehot = data_onehot.drop('Exited', axis=1)
y_onehot = data_onehot['Exited']

# Дерево

In [ ]:
from scipy.stats import bootstrap

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
class RandomForest:
  def __init__(self, N_trees=10, max_depth=5, max_samples_leaf=5, min_list_amount=5):
    self.N_trees = N_trees
    self.max_depth = max_depth
    self.min_list_amount=min_list_amount
    self.max_samples_leaf=max_samples_leaf
    self.trees = []

  def _bootstrap_split(self, X, y):
    idxs = np.random.choice(X.index, X.shape[0], replace=True)
    return X.loc[idxs], y.loc[idxs]

  def fit(self, X, y):
    self.trees = []
    for _ in range(self.N_trees):
      tree = TreeClassifier(max_depth=self.max_depth,
                      min_list_amount=self.min_list_amount,
                      max_samples_leaf=self.max_samples_leaf)
      # tree = DecisionTreeClassifier(criterion = 'entropy',
      #                               max_depth = self.max_depth,
      #                               min_samples_split = self.min_samples_split)
      X_sample, y_sample = self._bootstrap_split(X, y)
      tree.fit(X_sample, y_sample)
      self.trees.append(tree)

  def predict(self, X_test):
    tree_preds = np.swapaxes(np.array([tree.predict(X_test) for tree in self.trees]), 0, 1)
    predictions = np.array([np.round(np.mean(pred)) for pred in tree_preds])
    return predictions

#### Сравнение

In [ ]:
X_label

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,0,1,42,2,0.00,1,1,1,101348.88
1,608,2,1,41,1,83807.86,1,0,1,112542.58
2,502,0,1,42,8,159660.80,3,1,0,113931.57
3,699,0,1,39,1,0.00,2,0,0,93826.63
4,850,2,1,43,2,125510.82,1,1,1,79084.10
...,...,...,...,...,...,...,...,...,...,...
9995,771,0,0,39,5,0.00,2,1,0,96270.64
9996,516,0,0,35,10,57369.61,1,1,1,101699.77
9997,709,0,1,36,7,0.00,1,0,1,42085.58
9998,772,1,0,42,3,75075.31,2,1,0,92888.52


In [ ]:
X_train_l, X_test_l, y_train_l, y_test_l = train_test_split(X_label, y_label, test_size=0.3, random_state=1)

X_train_o, X_test_o, y_train_o, y_test_o = train_test_split(X_onehot, y_onehot, test_size=0.3, random_state=1)

In [ ]:
forest = RandomForest()
forest.fit(X_train_l, y_train_l)

In [ ]:
y_l_preds = forest.predict(X_test_l)
accuracy_score(y_test_l, y_l_preds)

0.852

In [ ]:
forest.fit(X_train_o, y_train_o)
y_o_preds = forest.predict(X_test_o)
accuracy_score(y_test_o, y_o_preds)

0.8573333333333333

OneHotEncoder сработал лучше, поэтому информативность признаков будем оценивать исходя из этой модели 🥸

In [ ]:
from sklearn.metrics import mean_squared_error
base = mean_squared_error(y_o_preds, y_test_o)

In [ ]:
scores={}
for col in X_test_o.columns:
    X = X_test_o.copy()
    X[col] = np.random.choice(X[col], len(X))
    scores[col]=mean_squared_error(forest.predict(X), y_test_o)-base

In [ ]:
fi = sorted(scores.items(), key=lambda x: x[1], reverse=True)
pd.DataFrame(fi, columns = ['feature', 'importance'])

,feature,importance
0,Age,0.078000
1,NumOfProducts,0.050333
2,IsActiveMember,0.027667
3,g2,0.011000
4,Balance,0.010333
5,Tenure,0.003667
6,EstimatedSalary,0.003333
7,Gender,0.001667
8,g3,0.001667
9,g1,0.001333


Как можно заметить, самыми важными признаками являются возраст клиента, количество продуктов, купленных через банк и степень активности клиента
